# Deep Learning Playground

We are interested in the following features:

- H1: region-averaged time-series
- H2: region-level connectivities (from H1, optional: triu-k1)
- H3: network-averaged time-series (from H1)
- H4: network connectivity (from H3, optional: triu-k1)
- H5: networks connectivity (from H2, optional: triu-k0)

> Note that we are not going to take the upper triangular part of the connectivity matrix and full matrices are used instead.

In [27]:
%reload_ext autoreload
%autoreload 2

from src.acnets.deep import ACNetsDataModule


In [28]:
atlases = [
    'dosenbach2010',
    'dosenbach2007',
    'difumo_64_2mm',
    'difumo_128_2mm',
    'difumo_256_2mm',
    'difumo_512_2mm',
    'difumo_1024_2mm',
    'gordon2014_2mm',
    'cort-maxprob-thr25-2mm',
    'seitzman2018',
    'friedman2020',
]

kinds = [
    'correlation',
    'partial correlation',
    'tangent',
    'covariance',
    'precisions'
]

In [49]:

datamodule = ACNetsDataModule(atlas='dosenbach2010', kind='partial correlation')
datamodule.prepare_data()

# for d in datamodule.train_dataloader().dataset:
#     print('grp', datamodule.y_encoder.inverse_transform([d[5].item()]).item())